In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18
np.random.seed(123)

In [24]:
df = pd.read_csv('./data/ch6_p5.csv', header=None)
df.columns = ['weights']

In [60]:
a = np.array(df).reshape(20,2, order='F')
df = pd.DataFrame(a, columns=['before', 'after'])

In [69]:
stan_model = """
    data{
        int N;
        vector<lower=0>[2] X[N];
    }
    
    parameters{
        vector[2] mu;
        vector<lower=0>[2] sigma;
        real<lower=-1,upper=1>    rho;
    }
    
    transformed parameters{
        vector<lower=0>[2] sigmasq;
        matrix[2,2] Sigma;

        sigmasq[1] = pow(sigma[1], 2);
        sigmasq[2] = pow(sigma[2], 2);
        Sigma[1,1] = sigmasq[1];
        Sigma[2,2] = sigmasq[2];
        Sigma[1,2] = sigma[1] * sigma[2] * rho;
        Sigma[2,1] = sigma[1] * sigma[2] * rho;

    }
    
    model{
        for(n in 1:N){
            X[n] ~ multi_normal(mu, Sigma);
        }
    }
    
    generated quantities{
        real mu_delta;
        real<lower=0, upper=1> mu_delta_0;
        real<lower=0, upper=1> mu_delta_2;
        
        mu_delta = -(mu[2] - mu[1]);
        mu_delta_0 <- step(mu_delta  - 0);
        mu_delta_2 <- step(mu_delta - 2);
    }
"""

In [70]:
sm = pystan.StanModel(model_code = stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d6c8c61026974599fda194058adcc750 NOW.


In [72]:
stan_data = {"N": df.shape[0], "X": df[['before', 'after']]}

In [73]:
fit = sm.sampling(data=stan_data, iter=11000, warmup=1000, chains=3, seed=1234)

/Users/yugi/.pyenv/versions/anaconda-4.0.0/envs/py36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [74]:
fit

Inference for Stan model: anon_model_d6c8c61026974599fda194058adcc750.
3 chains, each with iter=11000; warmup=1000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=30000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu[0]       51.65    0.01   1.23   49.2  50.85  51.65  52.47  54.07  14840    1.0
mu[1]       50.67    0.01   1.32  48.03  49.82  50.68  51.53  53.32  15042    1.0
sigma[0]     5.42  8.2e-3   0.95   3.95   4.75   5.29   5.95   7.62  13622    1.0
sigma[1]     5.82  8.7e-3   1.02   4.22   5.09   5.68   6.38   8.22  13750    1.0
rho          0.74  9.2e-4   0.11   0.47   0.68   0.76   0.82    0.9  14447    1.0
sigmasq[0]  30.33     0.1  11.34  15.57  22.59  28.01  35.46   58.0  13021    1.0
sigmasq[1]  34.86    0.11  12.88  17.81  25.95  32.21  40.72  67.57  13164    1.0
Sigma[0,0]  30.33     0.1  11.34  15.57  22.59  28.01  35.46   58.0  13021    1.0
Sigma[1,0]  24.34     0.1  10.42  10.26  17.22  22.33  29.11  50.23  1